Homework 02 

Royce Lloyd, 3/8/2019, CS344

In [1]:
"""
This module will create a sam filter based on Paul Graham’s A Plan for Spam.

Written by Royce Lloyd for CS 344 at Calvin College
3/8/2019
"""

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

ngood = len(spam_corpus)
nbad = len(ham_corpus)

emailContents = ["save", "on", "green", "eggs", "and", "spam", "do", "you", "like", "spam", "80", "$", "off", "click",
                 "here", "am", "I", "spamiam", "ham", "that", "i"]

def getprob(wordInQuestion):

    #Check for KeyErrors(Out of range of dict)
    try:
        g = (2 * goodHash[wordInQuestion])
    except KeyError:
        g = 0

    try:
        b = badHash[wordInQuestion]
    except KeyError:
        b = 0

    #Computer the possibility of spam
    if g + b > 0:
        return max(0.01, min(0.99, min(1.0, b/nbad) / (min(1.0, g/ngood) + min(1.0, b/nbad))))
    else:
        return 0

def isSpam(li):

    #Find prod
    prod = 1
    for value in li:
        prod = prod * value

    #Create a list f compliments
    complList = []
    for value in li:
        complList.append(1 - value)

    #Computer the product of those compliments
    compProd = 1
    for value in complList:
        compProd = compProd * value

    #Return the probability of the email being spam
    return prod / (prod + (compProd))


"""
This section takes our corpus(s) and creates a single list of tokens.

Big thank you to Alex Martelli on StackOverflow for lines 55-61
https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
"""
spamWords = []
hamWords = []

for sublist in spam_corpus:
    for item in sublist:
        spamWords.append(item)

for sublist in ham_corpus:
    for item in sublist:
        hamWords.append(item)

"""
Create a hash table(dictionary) for the number of times each word appears in the good and bad corpus
"""
goodHash = {}
badHash = {}

for word in hamWords:
    goodCount = 0
    for checkWord in hamWords:
        if word == checkWord:
            goodCount += 1
    goodHash[word] = goodCount


for word in spamWords:
    badCount = 0
    for checkWord in spamWords:
        if word == checkWord:
            badCount += 1
    badHash[word] = badCount


"""
Use the function getprob to determine the possibility that an email containing these words
is spam or not.
"""
hashTotal = {}

for word in goodHash:
    hashTotal[word] = getprob(word)

for word in badHash:
    hashTotal[word] = getprob(word)


"""
Create a list of probabilities from emailContents.
These probabilities are the chances of that word being used in a spam email,
based on our corpus.
"""
probsTemp = {}
probs = []

for word in emailContents:
    probsTemp[word] = getprob(word)

for key in probsTemp:
    if probsTemp[key] != 0:
        probs.append(probsTemp[key])




print("A higher number means spam, a lower number means ham.")
print("The probability of these words being spam, based on the spam and ham corpus, are:", "\n", hashTotal)
# print(probsTemp)
# print(probs)
print("The probability of the email, gives the contents of emailContents, is spam, is: ", isSpam(probs))


A higher number means spam, a lower number means ham.
The probability of these words being spam, based on the spam and ham corpus, are: 
 {'do': 0.3333333333333333, 'i': 0.01, 'like': 0.3333333333333333, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01, 'I': 0.99, 'am': 0.99, 'spam': 0.99, 'not': 0.99, 'that': 0.99, 'spamiam': 0.99}
The probability of the email, gives the contents of emailContents, is spam, is:  0.19999999999999918


This is a Bayesian approach because you are essentialy calculating the condition probability of an email being spam, given what you already know about the possibility of that word being spam. Grahm could create a Bayesian Network which describes how a word affects an emails chances of being spam. Your independat values would be the word itself. It would be directly connected to the email and have a probability calculated for it's chances of being spam.

In [2]:
"""
This module will compute a list of probabilities based on the Bayesian Network from Figure 14.12a

An extension of this exercise, meaning part 2b-d(handCalculations),
can be found in paper at the office of Prof. VanderLinden

Written by Royce Lloyd for CS344 at Calvin College
3/8/2019
"""

from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask


T, F = True, False

cloudy = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: .10, F: .50}),
    ('Rain', 'Cloudy', {T: .80, F: .20}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): .99, (T, F): .90, (F, T): .90, (F, F): .00})
])

#P(Sprinker | cloudy)
print(enumeration_ask('Sprinkler', dict(Cloudy=T), cloudy).show_approx())

#P(Cloudy| the sprinkler is running and it’s not raining)
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())

#P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())

#P(Cloudy | the grass is not wet)
print(enumeration_ask('Cloudy', dict(WetGrass=F), cloudy).show_approx())


False: 0.9, True: 0.1
False: 0.952, True: 0.0476
False: 0.01, True: 0.99
False: 0.639, True: 0.361
